In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("googleplaystore.csv")
df

In [ ]:
df.info()

In [ ]:
df["App"].value_counts() # Some app's have duplicated.
df = df.sort_values("Reviews" , ascending=False ).drop_duplicates(subset = "App") # We dropped same app name rows. We kept only most Reviews.
df.reset_index(drop = True ,inplace=True)

In [ ]:
df.Reviews = df.Reviews.str.replace(".0M","000000")
df.Reviews = df.Reviews.astype("int32")

In [ ]:
df.Size = df.Size.apply(lambda x : str(float(x.replace("M", ""))*1e6) if x.__contains__("M") else x)
df.Size = df.Size.apply(lambda x : str(float(x.replace("k", ""))*1e3) if x.__contains__("k") else x)
df.Size = df.Size.apply(lambda x : np.nan if x == "Varies with device" else x)
df.Size = df.Size.apply(lambda x : x.replace(",","") if str(x).__contains__(",") else x)
df.Size = df.Size.apply(lambda x : str(float(x.replace("+",""))*1.5) if str(x).__contains__("+") else x)
df.Size = df.Size.astype("float32")

In [ ]:
df.info()

In [ ]:
df[df["Installs"]== "Free"] # 1 cell shifted value.

In [ ]:
df.iloc[4484]

In [ ]:
df.iloc[4484, 1:] = np.roll(df.iloc[4484, 1:] , 1) # We solved the shifting problem.

In [ ]:
df.iloc[4484] # Yeah we done.

In [ ]:
df.Category = df.Category.astype("category")

In [ ]:
df.Rating = df.Rating.astype("float32")

In [ ]:
df.Installs = df.Installs.str.replace("," , "")
df.Installs = df.Installs.dropna().apply(lambda x : str(float(x.replace("+" , "")) *1.5) if x.__contains__("+") else x )
df.Installs = df.Installs.astype("float32")

df["Installs"].value_counts() 

In [ ]:
df.info()

In [ ]:
df["Type"] = df["Type"].astype("category")

In [ ]:
df.Price = df.Price.str.replace("$" , "").astype("float32")

In [ ]:
df["Content Rating"] = df["Content Rating"].astype("category")

In [ ]:
df["Last Updated"] = df["Last Updated"].astype("datetime64")

In [ ]:
#df["Current Ver"].value_counts() # Its very hard to change.
#df["Android Ver"].value_counts() # Same.

In [ ]:
df.info()

## Question 1 : Which ones are most Installed apps ?

In [ ]:
df[df.Installs == df.Installs.max()]

## Question 2 : Which ones are least Installed apps ?

In [ ]:
df[df.Installs == df.Installs.min()]

## Question 3 : Which Category has the most Installing?

In [ ]:
df.groupby("Category").count().sort_values("Size" , ascending=False) # 1879 Family , 942 Game , 828 Tools apps.

In [ ]:
df.groupby("Category").mean()["Installs"].sort_values(ascending=False) # Communication category have most Installing rates.

In [ ]:
df.groupby("Category").sum()["Installs"].sort_values(ascending=False) # Game Category most Installed. But also too many game apps...

In [ ]:
df.groupby("Category").mean()["Installs"].sort_values(ascending=False).plot(kind = "bar" , figsize = (20,15));

## Question 4 : Which categories get the highest ratings? 

In [ ]:
df.groupby("Category").mean()["Rating"].sort_values(ascending=False) # Events , Art and Design , Education , Book , Personalization ...

In [ ]:
df.groupby("Category").mean()["Rating"].sort_values(ascending=False).plot(kind = "bar" , figsize = (18,12));

## Question 5 : Is there any realation between app size and app installing ?

In [ ]:
df.info()

In [ ]:
df.corr()["Size"]["Installs"] # 0.13 correlation is too low. So there is no realation between size and installing.

## Question 6 : Lets comparate installing rates between free and paid apps.

In [ ]:
df.Type.value_counts() # First of all 8903 free , 756 paid app in our data.

In [ ]:
df.groupby("Type").mean() # We saw that Free apps too many more Installs and Reviews than Paid ones.
                          # But about ratings, paid apps have more ratings rate.

In [ ]:
fig,ax = plt.subplots(1,3 , figsize = (15,10))

sns.barplot(x = df["Type"] , y = df["Installs"] , ax = ax[0]);
sns.scatterplot(x = df["Price"] , y = df["Installs"] , ax = ax[1] );
sns.lineplot(x = df["Price"] , y = df["Installs"] , ax = ax[2] );

# We see that Free apps too many more installing rate than paid ones.

## Question 7 : Which content rating have more installing ?

In [ ]:
df.head()

In [ ]:
df.groupby("Content Rating").mean()["Installs"].sort_values(ascending=False) # Teens content rating have most installing rates. After teens, Everyone 10+ most popular content.

## Question 8 : Lets comparate categories and genres columns, are we need to keep both of them or can we delete someone ?

In [ ]:
df.groupby("Category")["Genres"].value_counts() # Best way to compare 2 feature.
                                                # Category looks like a main feature and Genres its subset. I'm not sure am i drop one of them. 
                                                # If i was going to build a model i would drop one, but I wont build model for now. So I won't drop anyone.

## Question 9 : 

## Question 10 : Lets try to find estimated number of users via Install's and Ratings.

In [ ]:
df.head()

In [ ]:
percentageUser = df["Rating"] * 20 / 100
df["Estimated Num of Users"] = df["Installs"] *percentageUser
df.head()

## Question 11: Find the realation between ratings and reviews by graphs.

In [ ]:
df.corr()["Rating"]["Reviews"] # 0.05 there is no any realation between rating and reviews.

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (15,12))

sns.scatterplot(x = df["Rating"] , y = df["Reviews"] , ax = ax[0]);
sns.lineplot(x = df["Rating"] , y = df["Reviews"] , ax = ax[1]);
sns.barplot(x = df["Rating"] , y = df["Reviews"] , ax = ax[2]);

# Graphs a little bit misleading. There is no good realation between Rating and Reviews.

## Question 12 : If you were to produce an app, what kind of app would you produce for more views?

In [ ]:
# If I were to make an app, It would be in the game or education category and it would be for teens and free app.